In [1]:
import sagemaker
import boto3 ## used to connect the sagemaker to the s3 bucket
from sklearn.model_selection import train_test_split
import pandas as pd

sm_boto3 = boto3.client('sagemaker', region_name='us-east-2')
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'aws-machine-learning-adeola'
print('Using bucket ' + bucket)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\adeol\AppData\Local\sagemaker\sagemaker\config.yaml
Using bucket aws-machine-learning-adeola


In [2]:
df = pd.read_csv('s3://aws-machine-learning-adeola/energy_price_data/Data.csv')

In [3]:
df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,Class
0,-2.30325,15.4530,853.76,-490.50,0.06,-11.2440,-1400.762,-22435.144,1535.900,-3.41000,...,9.8594,1,-5956.56,1.4626,-22.547,-9.2880,1.50200,-7870.52,16.02,True
1,-3.79380,8.2308,1077.72,-2340.20,2.06,-6.6428,-949.740,-22459.770,-562.100,-0.52780,...,15.5520,1,-5927.34,1.0864,-20.974,-13.7370,0.47138,-15984.08,NaN,False
2,-4.21470,7.7925,-4559.32,90.20,2.06,-6.9420,-1814.340,-22446.333,1238.500,-0.19285,...,12.4780,1,-5000.22,3.8200,-11.028,-7.3965,0.96080,-8012.68,NaN,True
3,-2.95410,7.5495,518.68,-565.70,0.06,-6.7114,-1346.520,-22414.960,1084.376,-0.53640,...,9.0148,1,-3457.32,5.0020,-25.280,-6.6633,1.04010,-8243.48,13.92,False
4,-2.67390,7.1814,891.36,-379.55,0.06,-10.6000,-1062.340,-22417.580,1016.740,-2.61700,...,10.1046,1,-5110.32,7.9840,-13.402,-8.6394,0.66500,-9224.08,16.16,True


In [4]:
df.isnull().sum()

F1         0
F2         0
F3         0
F4         0
F5         0
F6         0
F7         0
F8         0
F9         0
F10        0
F11        0
F12        0
F13        0
F14        0
F15        0
F16        0
F17        0
F18        0
F19        0
F20        0
F21      500
Class      0
dtype: int64

In [5]:
df['F21'] = df['F21'].fillna(df['F21'].mode())

In [6]:
df.isnull().sum()

F1         0
F2         0
F3         0
F4         0
F5         0
F6         0
F7         0
F8         0
F9         0
F10        0
F11        0
F12        0
F13        0
F14        0
F15        0
F16        0
F17        0
F18        0
F19        0
F20        0
F21      500
Class      0
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
features=list(df.columns)
label = features.pop(-1)

In [9]:
X = df[features]
y = df[label]

In [10]:
y.head()

0     True
3    False
4     True
7    False
8     True
Name: Class, dtype: bool

In [11]:
"""Replace True with 1 and False with 0 in y"""
y = y.replace({True: 1, False: 0})

C:\Users\adeol\AppData\Local\Temp\ipykernel_15620\3640143397.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({True: 1, False: 0})


In [12]:
y.value_counts()

Class
1    251
0    249
Name: count, dtype: int64

The data is balanced

Splitting the data into training ans test sets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
print('The shape of X_train is:', X_train.shape)
print('The shape of X_test is:', X_test.shape)
print('The shape of y_train is:', y_train.shape)
print('The shape of y_test is:', y_test.shape)


The shape of X_train is: (400, 21)
The shape of X_test is: (100, 21)
The shape of y_train is: (400,)
The shape of y_test is: (100,)


In [15]:
trainX = X_train
trainX[label] = y_train
testX = X_test
testX[label] = y_test

In [16]:
trainX.to_csv('train_data.csv', index=False)
testX.to_csv('test_data.csv', index=False)

Sending the data to S3 because Sagemaker will take data from S3

In [17]:
sk_prefix = "energy_price_data/sagemaker/energy_price_prediction/sklearncontainer"
trainpath = sess.upload_data(
    path='train_data.csv', 
    bucket=bucket,
    key_prefix=sk_prefix
    )

testpath = sess.upload_data(
    path='test_data.csv', 
    bucket=bucket,
    key_prefix=sk_prefix
    )

print('The path of the training data is:', trainpath)
print('The path of the test data is:', testpath)


The path of the training data is: s3://aws-machine-learning-adeola/energy_price_data/sagemaker/energy_price_prediction/sklearncontainer/train_data.csv
The path of the test data is: s3://aws-machine-learning-adeola/energy_price_data/sagemaker/energy_price_prediction/sklearncontainer/test_data.csv


In [21]:
%%writefile script.py
import argparse
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report, confusion_matrix
import joblib
import sklearn
import boto3
import pathlib
from io import StringIO
import numpy as np



def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__=="__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=100)

    #Data, model, and output directories
    parser.add_argument('--output-data-dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default='train_data.csv')
    parser.add_argument('--test-file', type=str, default='test_data.csv')

    args, _ = parser.parse_known_args()

    print("Sklearn version is:", sklearn.__version__)
    print("Joblib version is:", joblib.__version__)

    print("[INFO] Reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets")
    ()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print("Column order ")
    print(features)
    print()

    #print the shape of the training and testing datasets
    print("The shape of X_train is:", X_train.shape)
    print("The shape of X_test is:", X_test.shape)
    print("The shape of y_train is:", y_train.shape)
    print("The shape of y_test is:", y_test.shape)

    print("Training the RadomForest Model")
    print()
    clf = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state)
    clf.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    print("Saving the model")
    joblib.dump(clf, model_path)
    print("Model has been persisted at:", model_path)
    print()

    print("METRIC RESULTS FOR TESTING DATA")
    print()
    y_pred_test = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    print("[TESTING] Model Accuracy is:", test_acc)
    print("[TESTING] Model Precision is:", test_precision)
    print("[TESTING] Model Recall is:", test_recall)
    print("[TESTING] Model F1 Score is:", test_f1)
    print('[TESTING] Model Classification Report is', classification_report(y_test, y_pred_test))
    print()


Overwriting script.py


In [22]:
from sagemaker.sklearn.estimator import SKLearn
FRAMWORK_VERSION = '0.23-1'
sklearn_estimator = SKLearn(entry_point='script.py',
                                role='arn:aws:iam::737555234178:role/service-role/AmazonSageMaker-ExecutionRole-20240319T163960',
                                instance_count=1,
                                instance_type='ml.m5.large',
                                framework_version=FRAMWORK_VERSION,
                                sagemaker_session=sess,
                                base_job_name='energy-price-prediction-sklearn',
                                hyperparameters={'n_estimators': 100, 'random_state': 0},
                                use_spot_instances=True,
                                max_run=3600,
                                max_wait=7200)

  

In [23]:
#Launching training job
print("Launching training job")
sklearn_estimator.fit({'train': trainpath, 'test': testpath}, wait = True)


Launching training job


INFO:sagemaker:Creating training-job with name: energy-price-prediction-sklearn-2024-03-23-02-30-08-528


2024-03-23 02:30:10 Starting - Starting the training job...
2024-03-23 02:30:25 Starting - Preparing the instances for training......
2024-03-23 02:31:16 Downloading - Downloading input data...
2024-03-23 02:31:47 Downloading - Downloading the training image..2024-03-23 02:32:21,457 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-03-23 02:32:21,460 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-03-23 02:32:21,505 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-03-23 02:32:21,748 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-03-23 02:32:21,760 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-03-23 02:32:21,771 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-03-23 02:32:21,780 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    

In [25]:
sklearn_estimator.latest_training_job.wait(logs='None')
artifact = sm_boto3.describe_training_job(
            TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']
print('The model artifact is persisted at:', artifact)


2024-03-23 02:32:38 Starting - Preparing the instances for training
2024-03-23 02:32:38 Downloading - Downloading the training image
2024-03-23 02:32:38 Training - Training image download completed. Training in progress.
2024-03-23 02:32:38 Uploading - Uploading generated training model
2024-03-23 02:32:38 Completed - Training job completed
The model artifact is persisted at: s3://sagemaker-us-east-2-737555234178/energy-price-prediction-sklearn-2024-03-23-02-30-08-528/output/model.tar.gz


Deploying the model

In [26]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "energy-price-prediction-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data=artifact,
    role='arn:aws:iam::737555234178:role/service-role/AmazonSageMaker-ExecutionRole-20240319T163960',
    entry_point='script.py',
    framework_version=FRAMWORK_VERSION
)


In [27]:
model

In [28]:
##Endpoint Deployment
endpoint_name = 'energy-price-prediction-sklearn-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('EndPointName = {}'.format(endpoint_name))

predict = model.deploy(
    endpoint_name=endpoint_name,
    instance_type='ml.m5.large',
    initial_instance_count=1
)

EndPointName = energy-price-prediction-sklearn-endpoint-2024-03-23-17-28-39


INFO:sagemaker:Creating model with name: energy-price-prediction-sklearn-model-2024-03-23-17-25-43
INFO:sagemaker:Creating endpoint-config with name energy-price-prediction-sklearn-endpoint-2024-03-23-17-28-39
INFO:sagemaker:Creating endpoint with name energy-price-prediction-sklearn-endpoint-2024-03-23-17-28-39


------!